In [1]:
import pandas as pd

df = pd.read_parquet('../test/')

In [2]:
df.head()

,user_id,merchant_abn,dollar_value,order_id,order_datetime
0,1,28000487688,133.226894,0c37b3f7-c7f1-48cb-bcc7-0a58e76608ea,2021-02-28
1,18485,62191208634,79.131400,9e18b913-0465-4fd4-92fd-66d15e65d93c,2021-02-28
2,1,83690644458,30.441348,40a2ff69-ea34-4657-8429-df7ca957d6a1,2021-02-28
3,18488,39649557865,962.813341,f4c1a5ae-5b76-40d0-ae0f-cb9730ac325a,2021-02-28
4,2,80779820715,48.123977,cd09bdd6-f56d-489f-81ea-440f4bda933c,2021-02-28


In [3]:
merchant = pd.read_parquet('../data/curated/merchant_info.parquet')

In [4]:
merchant.head()

,merchant_abn,name,category,revenue_rate,fraud_probability,take_rate_value
0,10023283211,Felis Limited,"furniture, home furnishings and equipment shop...",e,0.0,0.18
1,10142254217,Arcu Ac Orci Corporation,"cable, satellite, and otHer pay television and...",b,0.0,4.22
2,10165489824,Nunc Sed Company,"jewelry, watch, clock, and silverware shops",b,0.0,4.40
3,10187291046,Ultricies Dignissim Lacus Foundation,"wAtch, clock, and jewelry repair shops",b,0.0,3.29
4,10192359162,Enim Condimentum PC,"music shops - musical instruments, pianos, and...",a,0.0,6.33


In [5]:
categories = pd.read_parquet('../data/curated/merchant_categories.parquet')

In [6]:
categories.head()   

,merchant_abn,name,category,revenue_rate,fraud_probability,take_rate_value,category_label
0,10023283211,Felis Limited,"furniture, home furnishings and equipment shop...",e,0.0,0.18,Furniture and Home Furnishing
1,10142254217,Arcu Ac Orci Corporation,"cable, satellite, and otHer pay television and...",b,0.0,4.22,"Media and Technology, Services"
2,10165489824,Nunc Sed Company,"jewelry, watch, clock, and silverware shops",b,0.0,4.40,Retail Goods
3,10187291046,Ultricies Dignissim Lacus Foundation,"wAtch, clock, and jewelry repair shops",b,0.0,3.29,"Retail Goods, Services"
4,10192359162,Enim Condimentum PC,"music shops - musical instruments, pianos, and...",a,0.0,6.33,Retail Goods


In [7]:
# Ensure the 'order_datetime' column is in datetime format
df['order_datetime'] = pd.to_datetime(df['order_datetime'])

# Set 'order_datetime' as the index
df.set_index('order_datetime', inplace=True)

# Convert the index to a DatetimeIndex
df.index = pd.DatetimeIndex(df.index)

# Perform the groupby and resample operation to calculate the mean dollar value and count of transactions
grouped = df.groupby('merchant_abn').resample('M').agg({
    'dollar_value': 'mean',
    'merchant_abn': 'count'
}).rename(columns={'merchant_abn': 'transaction_count'}).reset_index()

# Display the result
grouped.head()

C:\Users\luisc\AppData\Local\Temp\ipykernel_10672\934546246.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  grouped = df.groupby('merchant_abn').resample('M').agg({


,merchant_abn,order_datetime,dollar_value,transaction_count
0,10023283211,2021-02-28,233.855546,3
1,10023283211,2021-03-31,221.931041,111
2,10023283211,2021-04-30,214.126688,129
3,10023283211,2021-05-31,209.110300,144
4,10023283211,2021-06-30,198.554719,145


In [8]:
# Merge grouped with categories by 'merchant_abn', including 'category_label', 'take_rate_value', and 'fraud_probability'
merged = pd.merge(grouped, categories[['merchant_abn', 'category_label', 'take_rate_value', 'fraud_probability']], on='merchant_abn', how='left')

# Display the result
merged.head()

,merchant_abn,order_datetime,dollar_value,transaction_count,category_label,take_rate_value,fraud_probability
0,10023283211,2021-02-28,233.855546,3,Furniture and Home Furnishing,0.18,0.0
1,10023283211,2021-03-31,221.931041,111,Furniture and Home Furnishing,0.18,0.0
2,10023283211,2021-04-30,214.126688,129,Furniture and Home Furnishing,0.18,0.0
3,10023283211,2021-05-31,209.110300,144,Furniture and Home Furnishing,0.18,0.0
4,10023283211,2021-06-30,198.554719,145,Furniture and Home Furnishing,0.18,0.0


In [9]:
# Calculate the average monthly revenue by merchant_abn
merged['average_monthly_revenue'] = merged.groupby('merchant_abn')['dollar_value'].transform('mean')

# Calculate the average transaction count by merchant_abn
merged['average_monthly_transactions'] = merged.groupby('merchant_abn')['transaction_count'].transform('mean')

# Drop duplicates to keep only one row per merchant_abn
merged = merged.drop_duplicates(subset='merchant_abn')

# Drop unnecessary columns
merged = merged.drop(columns=['dollar_value', 'order_datetime', 'transaction_count'])

# Display the result
merged.head()

,merchant_abn,category_label,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions
0,10023283211,Furniture and Home Furnishing,0.18,0.0,216.736529,155.285714
21,10142254217,"Media and Technology, Services",4.22,0.0,37.716001,144.571429
42,10165489824,Retail Goods,4.40,0.0,11236.094771,0.454545
53,10187291046,"Retail Goods, Services",3.29,0.0,119.526004,16.800000
73,10192359162,Retail Goods,6.33,0.0,462.329450,19.250000


In [10]:
merged.to_parquet('../data/curated/merged_merchant_info_with_categories.parquet')

In [11]:
merged['category_label'] = merged['category_label'].fillna('other')
merged = merged.drop_duplicates(subset='merchant_abn')
merged = merged.drop(columns=['merchant_abn'])

In [12]:
# Define the categories
categories2 = ['Retail Goods', 'Media and Technology', 'Furniture and Home Furnishing', 'Services', 'Automotives']

# Create new columns for each category
for category in categories2:
    merged[category] = merged['category_label'].str.contains(category).astype(int)

# Drop the original category_label column
merged = merged.drop(columns=['category_label'])

# Display the result
merged.head()

,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Retail Goods,Media and Technology,Furniture and Home Furnishing,Services,Automotives
0,0.18,0.0,216.736529,155.285714,0,0,1,0,0
21,4.22,0.0,37.716001,144.571429,0,1,0,1,0
42,4.40,0.0,11236.094771,0.454545,1,0,0,0,0
53,3.29,0.0,119.526004,16.800000,1,0,0,1,0
73,6.33,0.0,462.329450,19.250000,1,0,0,0,0


In [13]:
# Replace null values in 'take_rate_value' with the mean of 'take_rate_value'
take_rate_mean = merged['take_rate_value'].mean()
merged['take_rate_value'] = merged['take_rate_value'].fillna(take_rate_mean)

# Replace null values in 'fraud_probability' with 0
merged['fraud_probability'] = merged['fraud_probability'].fillna(0)

# Display the result
merged.head()

,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Retail Goods,Media and Technology,Furniture and Home Furnishing,Services,Automotives
0,0.18,0.0,216.736529,155.285714,0,0,1,0,0
21,4.22,0.0,37.716001,144.571429,0,1,0,1,0
42,4.40,0.0,11236.094771,0.454545,1,0,0,0,0
53,3.29,0.0,119.526004,16.800000,1,0,0,1,0
73,6.33,0.0,462.329450,19.250000,1,0,0,0,0


In [14]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming merged DataFrame is already prepared

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Display the coefficients
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Coefficients: [-5.44545327e+01  8.46834800e+01 -3.40458742e-01 -1.76481919e+03
 -1.69717577e+03 -2.43207598e+03 -1.37328665e+03 -1.45589094e+03]
Intercept: 4043.0799107250104


In [15]:
merged.head()

,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Retail Goods,Media and Technology,Furniture and Home Furnishing,Services,Automotives
0,0.18,0.0,216.736529,155.285714,0,0,1,0,0
21,4.22,0.0,37.716001,144.571429,0,1,0,1,0
42,4.40,0.0,11236.094771,0.454545,1,0,0,0,0
53,3.29,0.0,119.526004,16.800000,1,0,0,1,0
73,6.33,0.0,462.329450,19.250000,1,0,0,0,0


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the accuracy (R^2 score)
accuracy = model.score(X_test, y_test)
print("Accuracy (R^2 score):", accuracy)

# Display the coefficients
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Mean Squared Error: 14953825.419167014
Accuracy (R^2 score): 0.12020198784984493
Coefficients: [-9.21444384e+01  7.44655218e+01 -3.42930453e-01 -1.69656709e+03
 -1.65195861e+03 -2.37066177e+03 -1.37739711e+03 -1.43512946e+03]
Intercept: 4126.228803781954


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Train the Random Forest Regressor
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the accuracy (R^2 score)
accuracy = model.score(X_test, y_test)
print("Accuracy (R^2 score):", accuracy)

# Display the feature importances
print("Feature Importances:", model.feature_importances_)

Mean Squared Error: 10285010.641676921
Accuracy (R^2 score): 0.3948884874707468
Feature Importances: [0.2039988  0.05325143 0.62409637 0.04509382 0.00198282 0.03204848
 0.0350872  0.00444109]


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

# Train the Gradient Boosting Regressor
model = GradientBoostingRegressor(
    learning_rate=0.01,
    max_depth=5,
    min_samples_leaf=1,
    min_samples_split=10,
    n_estimators=300
)

model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the accuracy (R^2 score)
accuracy = model.score(X_test, y_test)
print("Accuracy (R^2 score):", accuracy)

# Display the feature importances
print("Feature Importances:", model.feature_importances_)

Mean Squared Error: 6484949.740066742
Accuracy (R^2 score): 0.38491101000736594
Feature Importances: [0.08152354 0.11935762 0.64705382 0.04990952 0.00177757 0.05638174
 0.03866847 0.00532771]


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Grid Search
grid = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)
grid.fit(X_train, y_train)

# Predict on the test set
y_pred = grid.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the accuracy (R^2 score)
accuracy = grid.score(X_test, y_test)
print("Accuracy (R^2 score):", accuracy)

# Display the best parameters
print("Best Parameters:", grid.best_params_)

# Display the feature importances
print("Feature Importances:", grid.best_estimator_.feature_importances_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=

KeyboardInterrupt: 

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Train the Support Vector Regressor
model = SVR()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the accuracy (R^2 score)
accuracy = model.score(X_test, y_test)
print("Accuracy (R^2 score):", accuracy)

Mean Squared Error: 18344737.690363787
Accuracy (R^2 score): -0.0792999985616476


In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Train the K-Nearest Neighbors Regressor
model = KNeighborsRegressor()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the accuracy (R^2 score)
accuracy = model.score(X_test, y_test)
print("Accuracy (R^2 score):", accuracy)

Mean Squared Error: 8418735.9075552
Accuracy (R^2 score): 0.5046894751900358


In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Define the response variable
y = merged['average_monthly_revenue']

# Define the features (excluding the response variable and any non-numeric columns)
X = merged.drop(columns=['average_monthly_revenue'])

# Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Train the Neural Network Regressor
model = MLPRegressor(max_iter=10000, random_state=3)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate the accuracy (R^2 score)
accuracy = model.score(X_test, y_test)
print("Accuracy (R^2 score):", accuracy)

Mean Squared Error: 9651078.937993333
Accuracy (R^2 score): 0.4321854223423356
